In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tkinter import *
from PIL import ImageTk, Image
from tensorflow.keras import callbacks

C:\Users\kgesiak\Anaconda3\envs\OCR\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kgesiak\Anaconda3\envs\OCR\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kgesiak\Anaconda3\envs\OCR\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kgesiak\Anaconda3\envs\OCR\lib\site-packages\tensorflow\python\f

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [4]:
x_train = tf.keras.utils.normalize(x_train,axis = 1)
x_test = tf.keras.utils.normalize(x_test,axis = 1)

In [13]:
early_stopping = callbacks.EarlyStopping(
patience =500,
min_delta = 0.0005,
    restore_best_weights = True

)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(units=128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(units=10, activation = tf.nn.softmax))
          
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train,y_train,epochs=3,callbacks=[early_stopping])

loss, accuracy = model.evaluate(x_test,y_test)
print(accuracy)
print(loss)
          
model.save('digits.model')

Epoch 1/3
60000/60000 [==============================] - 3s 43us/sample - loss: 0.2640 - acc: 0.9232
Epoch 2/3
60000/60000 [==============================] - 3s 43us/sample - loss: 0.1079 - acc: 0.9661
Epoch 3/3
10000/10000 [==============================] - 0s 24us/sample - loss: 0.0876 - acc: 0.9740
0.974
0.08761529441140592


In [38]:
drawing = False # true if mouse is pressed
pt1_x , pt1_y = None , None

# mouse callback function
def line_drawing(event,x,y,flags,param):
    global pt1_x,pt1_y,drawing

    if event==cv2.EVENT_LBUTTONDOWN:
        drawing=True
        pt1_x,pt1_y=x,y

    elif event==cv2.EVENT_MOUSEMOVE:
        if drawing==True:
            cv2.line(img,(pt1_x,pt1_y),(x,y),color=255,thickness=12)
            pt1_x,pt1_y=x,y
    elif event==cv2.EVENT_LBUTTONUP:
        drawing=False
        cv2.line(img,(pt1_x,pt1_y),(x,y),color=255,thickness=3)        


img = np.zeros((280,280), np.uint8)
cv2.namedWindow('Draw a number')
cv2.setMouseCallback('Draw a number',line_drawing)

while(1):
    cv2.imshow('Draw a number',img)
    if cv2.waitKey(100) == 27:
        break
cv2.destroyAllWindows()

In [39]:
resized = cv2.resize(img, (28,28),interpolation = cv2.INTER_AREA)
resized = resized/255
resized = resized.reshape((1,28,28))

In [40]:
print("Predicted number:",np.argmax(model.predict(resized)))

Predicted number: 3
